In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import dask
import dask.dataframe as dd
from dask.distributed import Client, progress
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
client

import numpy as np
import warnings
import mlflow
import seaborn as sns

from modeling.config import EXPERIMENT_NAME
TRACKING_URI = open("../.mlflow_uri").read().strip()

warnings.filterwarnings('ignore')

2022-04-21 09:46:11,680 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/kaergell/Desktop/Neue Fische/Bootcamp/Capestone/fashion-recommender/notebooks/dask-worker-space/worker-7jcyova8', purging
2022-04-21 09:46:11,681 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/kaergell/Desktop/Neue Fische/Bootcamp/Capestone/fashion-recommender/notebooks/dask-worker-space/worker-ahixvnbu', purging


In [11]:
articles = pd.read_csv("../data/articles.csv").sort_values ("article_id").reset_index(drop=True)
custom = pd.read_csv("../data/customers.csv").sort_values("customer_id").reset_index(drop=True)
trans= pd.read_csv("../data/transactions_train.csv").sort_values("customer_id").reset_index(drop=True)
sample=pd.read_csv("../data/sample_submission.csv").sort_values("customer_id").reset_index(drop=True)

## Articles Data

In [3]:
articles.columns

Index(['article_id', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')

In [4]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [5]:
articles.info(verbose=True)

<class 'dask.dataframe.core.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   article_id                    105542 non-null      int64
 1   product_code                  105542 non-null      int64
 2   prod_name                     105542 non-null      object
 3   product_type_no               105542 non-null      int64
 4   product_type_name             105542 non-null      object
 5   product_group_name            105542 non-null      object
 6   graphical_appearance_no       105542 non-null      int64
 7   graphical_appearance_name     105542 non-null      object
 8   colour_group_code             105542 non-null      int64
 9   colour_group_name             105542 non-null      object
10   perceived_colour_value_id     105542 non-null      int64
11   perceived_colour_value_name   105542 non-null      object
12   perceived_colour_m

In [9]:
articles.describe()

,article_id,product_code,product_type_no,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,department_no,index_group_no,section_no,garment_group_no
npartitions=1,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...


In [7]:
articles.isnull().sum()

Dask Series Structure:
npartitions=1
article_id    int64
section_no      ...
dtype: int64
Dask Name: dataframe-sum-agg, 6 tasks

|Column name| Datatyp| Meaning | nunique| annotation|
|----------|-------| ---- |----|---|
| [article_id](#article_id) 	|int64 | Id connected to image name| 105542| |
|prod_name	|object | overall product description | 47224| Takes the first 7 digits of the article ID|
|product_code	|int64| individual product name| 45875| |
|product_type_no	|int64 | product type number| 132| -1 = unknown = NaN ? = 121||
|product_type_name	|object | product type name| 131| 131 values + -1 for unknown |
|product_group_name	|object | Product typ categories by name|19| |
|graphical_appearance_no	| int64| Appearance no| 30| -1 = unknown = NaN ? = 52|
|graphical_appearance_name	|object| Appearance name| 30| |
|colour_group_code	|int64| (dominating)Colour group number|50| No closed numbers i.e. 18, 24-29, 34-39 etc. is missing|
|colour_group_name	|object| (dominating)Colour group name| 50| |
|perceived_colour_value_id	|int64| "gray" scale or lightning condition of the image| 8 | -1 = unknown = NaN? = 105542|
|perceived_colour_value_name	|object| description of the lightning condition| 8 | | |
|perceived_colour_master_id	|int64|color master id | 20 | 17 is missing and -1 = unknown|
|perceived_colour_master_name	|object|  naming of the color master id |20 | 
|department_no	|int64| Department number| 299| More than one department might be responsible for  a certain topic| 
|department_name	|object| Department namen (topics) |250| |
|index_code	|object| "target group classification" index code| 10| A-F|
|index_name	|object| "target group classification"  index name | 10 | Ladieswear, Menswear etc.| 
|index_group_no	|int64| Corse "target group classification"  |5| 1,2,3,4,26|
|index_group_name	|object| Corse "target group classification"  name | 5|  'Ladieswear' = 1 ,'Divided' =2,'Menswear'=3 'Baby/Children' = 4 , 'Sport' =26|
|section_no	|int64| "target group classification" + usage numbering| 57| not a closed numbering|
|section_name	|object| "target group classification" + usage naming| 56| 1 is missing|
|garment_group_no	|int64| co|
|garment_group_name	|object|
|detail_desc	|object|

### article_id